<a href="https://colab.research.google.com/github/Ekliipce/Machine-Learning-for-Biomedical/blob/pre-processing/eeg/EEG_and_alcohol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electroencephalogram (EEG) and alcohol


## **EEG**
#### **What is EEG ?**
An electroencephalogram (EEG) is a test that records the brain's electrical activity noninvasively through electrodes placed on the scalp. The procedure involves placing these electrodes that are connected by wires to a computer, which then records and analyzes the electrical impulses in the brain. EEG is used for diagnosing and managing brain-related disorders like epilepsy, monitoring brain activity during surgeries, and conducting neuroscience research.

EEG patterns, consisting of different waves, are analyzed to understand normal or abnormal brain function. The procedure is safe, though preparation is required, and it might be slightly uncomfortable. EEG primarily detects activity in the brain's cortex with limited spatial resolution and can be affected by various factors like age and medication. Unlike MRI and CT scans that visualize brain structure, EEG captures real-time activity, making it a valuable tool in neuroscience and medicine.
<br><br>
#### **What does an EEG help diagnose?**

EEG is used primarily to diagnose conditions that affect brain activity. It’s particularly useful in identifying epilepsy and other seizure disorders by capturing the electrical activity of the brain. Besides, EEG can also help diagnose or manage other conditions like sleep disorders, depth of anesthesia, coma, encephalopathies, brain death, and certain psychiatric disorders. It is often used in conjunction with other diagnostic tools to provide comprehensive insights into brain health and function.
<br><br>

#### **What factors can influence the results of an EEG?**

Various factors can influence EEG results. Medications (such as sedatives, anti-epileptic drugs) can alter electrical activity in the brain, affecting the test's findings. The patient's age and overall brain development can also play a role in the results. The physical and mental state of the patient during the test, like being stressed, relaxed, asleep, or awake, can also influence the brain's electrical activity. External interference from electronic devices and not following preparatory instructions (like washing hair to ensure good electrode contact) can also impact the data quality and test outcomes.
<br><br>
#### **How reliable is EEG in diagnosing various brain disorders?**

EEG is a reliable tool for diagnosing disorders related to abnormal brain activity, like epilepsy. However, its reliability can be influenced by the technician's skill, the patient's cooperation, and the above-mentioned factors that might affect the results. While EEG provides valuable real-time data on brain function, it might not catch intermittent or infrequent abnormalities in brain activity if they don't occur during the test. Therefore, it's often used alongside other diagnostic methods, like MRI or CT scans, to provide a more complete picture of brain health and accurate diagnosis.


## **Brain and Alcohol**
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6668890/ searched deeper for Alcoholism and Human Electrophysiology.

Interestingly, the article suggests that the observed electrical abnormalities in the brains of alcoholics might not be a result of alcohol consumption per se, but rather a pre-existing condition, possibly serving as a risk marker for alcoholism. Some of these electrical characteristics, such as increased resting beta power and decreased active theta oscillations during cognitive tasks, have also been identified in individuals at high risk for developing alcoholism, even before any exposure to alcohol. Therefore, the text proposes that an inherent imbalance in CNS excitation and inhibition might predispose individuals to alcoholism. This imbalance is suggested to not only contribute to the risk of developing alcoholism but might also offer insights into the neurobiology of craving and relapse in alcoholism

## Dataset

In [118]:
!wget https://archive.ics.uci.edu/static/public/121/eeg+database.zip
!unzip eeg+database.zip
!tar -xf SMNI_CMI_TEST.tar.gz
!tar -xf SMNI_CMI_TRAIN.tar.gz
!gunzip -k SMNI_CMI_*/*/*.gz
!rm SMNI_CMI_*/*/*.gz
!mkdir train
!mkdir test

--2023-10-07 09:42:22--  https://archive.ics.uci.edu/static/public/121/eeg+database.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘eeg+database.zip.1’

eeg+database.zip.1      [    <=>             ] 762.44M  50.6MB/s    in 16s     

2023-10-07 09:42:38 (49.0 MB/s) - ‘eeg+database.zip.1’ saved [799481741]

Archive:  eeg+database.zip
replace SMNI_CMI_TEST.tar.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: gzip: SMNI_CMI_TEST/co2a0000364/co2a0000364.rd.030 already exists; do you wish to overwrite (y or n)? ^C
mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘test’: File exists


In [86]:
!pip install -U mne

In [87]:
import numpy as np

import mne
import os

from tqdm import tqdm

In [95]:
file = '/content/SMNI_CMI_TRAIN/co2a0000364/co2a0000364.rd.000'


def extract_raw_data(file):
  with open(file) as f:
    lines = f.readlines()

    data = []
    channel_names = []
    is_Alcolic = lines[0][5] == 'a'
    id_patient = int(lines[0][6:13])
    trial = int(lines[4].split()[-1])

    for line in lines[4:]:
      line_split = line.split()

      if (not line.startswith('#')):
        values = line_split[-1]
        data.append(values)
      else :
        ch_name = line_split[1]
        channel_names.append(ch_name)

    data = np.array(data, dtype="float").reshape((64, -1))
    info = {"his_id" : id_patient,  "is_Alcolic" : is_Alcolic,'id': id_patient, "trial": trial}
    return data, channel_names, info


data, channel_names, a = extract_raw_data(file)
data, channel_names, a

(array([[ -8.921,  -8.433,  -2.574, ...,   4.262,   5.727,   8.169],
        [  0.834,   3.276,   5.717, ...,   2.787,   9.623,  13.041],
        [-19.847, -12.522,   1.149, ...,   7.009,  11.403,  12.868],
        ...,
        [ -0.478,  -0.966,  -0.966, ...,   4.405,   3.916,   2.94 ],
        [ -8.901,  -7.924,  -3.042, ...,   4.771,   6.236,   8.677],
        [ -5.636,  -2.706,   1.689, ...,  15.849,  16.337,  14.872]]),
 ['FP1',
  'FP2',
  'F7',
  'F8',
  'AF1',
  'AF2',
  'FZ',
  'F4',
  'F3',
  'FC6',
  'FC5',
  'FC2',
  'FC1',
  'T8',
  'T7',
  'CZ',
  'C3',
  'C4',
  'CP5',
  'CP6',
  'CP1',
  'CP2',
  'P3',
  'P4',
  'PZ',
  'P8',
  'P7',
  'PO2',
  'PO1',
  'O2',
  'O1',
  'X',
  'AF7',
  'AF8',
  'F5',
  'F6',
  'FT7',
  'FT8',
  'FPZ',
  'FC4',
  'FC3',
  'C6',
  'C5',
  'F2',
  'F1',
  'TP8',
  'TP7',
  'AFZ',
  'CP3',
  'CP4',
  'P5',
  'P6',
  'C1',
  'C2',
  'PO7',
  'PO8',
  'FCZ',
  'POZ',
  'OZ',
  'P2',
  'P1',
  'CPZ',
  'nd',
  'Y'],
 {'his_id': 364, 'is_Alcolic'

In [91]:
def save_raw_files(file, save=False, train=True):
  file_name = file.split("/")[-1].replace(".", "_") + "_eeg"

  data, ch_names, info_patient = extract_raw_data(file)
  info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types='eeg')

  raw = mne.io.RawArray(data=data, info=info, verbose=False)
  raw.info['subject_info'] = info_patient
  print(f"{raw.info['subject_info'] = }")


  if (save):
    dir = "train" if train else "test"
    print(f"{dir}/{file_name}.fif")
    raw.save(f"{dir}/{file_name}.fif", overwrite=True, verbose=False)
  return raw



raw.info['subject_info'] = {'his_id': 364, 'is_Alcolic': True, 'id': 364, 'trial': 0}
train/co2a0000364_rd_000_eeg.fif
raw.info['subject_info'] = {'id': 364, 'his_id': '364'}


In [6]:
for dir_name, subdirs, files in tqdm(list(os.walk('/content/'))):
    for file_name in files:
      if ((".rd.") in file_name):
        current_file = os.path.join(dir_name, file_name)
        train = "TRAIN" in current_file
        save_raw_files(current_file, save=True, train=train)


100%|██████████| 51/51 [01:11<00:00,  1.40s/it]


## Pre-processing

In [ ]:
! find -name '*.fif'

In [111]:
file_data = mne.io.read_raw('./train/co2a0000372_rd_001_eeg.fif')
list(file_data.info.keys())
file_data.info['subject_info']

Opening raw data file ./train/co2a0000372_rd_001_eeg.fif...
Isotrak not found
    Range : 0 ... 255 =      0.000 ...     0.996 secs
Ready.


{'id': 372, 'his_id': '372'}

In [120]:
! ls /content

alcoholic.gif	    eeg_full	   SMNI_CMI_TEST	  test
control.gif	    eeg.full.html  SMNI_CMI_TEST.tar.gz   train
eeg+database.zip    eeg_full.tar   SMNI_CMI_TRAIN
eeg+database.zip.1  eeg.html	   SMNI_CMI_TRAIN.tar.gz
eeg.data.html	    sample_data    smni_eeg_data.tar.gz


In [127]:
data, channel_names,a =None,None,None
for dir_name, subdirs, files in tqdm(list(os.walk('/content/'))):
  for file_name in files:
    current_file = os.path.join(dir_name, file_name)
    if ((".rd.") in file_name):
      print(file_name)
      data, channel_names, a = extract_raw_data(current_file)
      break
  if a is not None:
    break
data, channel_names, a
info = a

 12%|█▏        | 6/51 [00:00<00:00, 91.54it/s]

co2c0000347.rd.071


In [125]:
import pandas as pd

In [132]:
df = None
data, channel_names, info = extract_raw_data(file)
for time_series,channel in zip(data, channel_names):

  if(df is None):
    df = pd.DataFrame({'value':time_series,'channel':channel,**info})
  else:
    cur_df = pd.DataFrame({'value':time_series,'channel':channel,**info})
    df = pd.concat([df,cur_df],ignore_index=True)
df

,value,channel,his_id,is_Alcolic,id,trial
0,-8.921,FP1,364,True,364,0
1,-8.433,FP1,364,True,364,0
2,-2.574,FP1,364,True,364,0
3,5.239,FP1,364,True,364,0
4,11.587,FP1,364,True,364,0
...,...,...,...,...,...,...
16379,6.571,Y,364,True,364,0
16380,12.431,Y,364,True,364,0
16381,15.849,Y,364,True,364,0
16382,16.337,Y,364,True,364,0
